In [1]:
import pandas as pd
from pandas import DataFrame
df=pd.read_csv("A1.csv",names=['id','label'])
df1_T=df[0:49]
df1_V=df[49:74]
training_set = pd.read_csv('A1.csv').values
#A=training_set[:,1]
#Train_L=A[0:49]
#print(df1_T)
#print(df1_V)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator = train_datagen.flow_from_dataframe(
        dataframe=df1_T,
        directory='A1_LCC/A1_T',
        x_col="id",
        y_col="label",
        shuffle=True,
        target_size=(320, 320),
        batch_size=6,
        class_mode="raw")
valid_generator=train_datagen.flow_from_dataframe(
                dataframe=df1_V,
                directory='A1_LCC/A1_V',
                x_col="id",
                y_col="label",
               
                batch_size=6,
                #seed=49,
                shuffle=True,
                class_mode="raw",
                target_size=(320,320))

Found 49 validated image filenames.
Found 25 validated image filenames.


In [3]:
from keras.applications.resnet50 import ResNet50
res_model = ResNet50(include_top=False,weights='imagenet',input_shape=(320,320,3))

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [4]:
from keras import Model, layers
from keras import regularizers
from keras.optimizers import Adam
#model=model.include_top=False
#model.summary()
#model=ResNet50()
#model.summary()
x = res_model.output
print(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu', activity_regularizer=regularizers.l2(0.2))(x)
leaf_pred =layers. Dense(1)(x)
model = Model(inputs = res_model.input, outputs = leaf_pred)
model.compile(optimizer=Adam(lr=0.0001), loss= 'mse',metrics=['accuracy'])

Tensor("activation_49/Relu:0", shape=(None, 10, 10, 2048), dtype=float32)


In [5]:
import keras
from keras.callbacks import EarlyStopping
csv_logger = keras.callbacks.CSVLogger('training.log', separator=',')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.03, mode='min', patience=8)

In [12]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_data=valid_generator,
                    validation_steps=step_size_valid,
                    #steps_per_epoch=1,
                    epochs=50,
                    callbacks= [csv_logger,early_stop])

Epoch 1/50
8/8 [==============================] - 6s 768ms/step - loss: 251.0970 - accuracy: 0.0000e+00 - val_loss: 278.1227 - val_accuracy: 0.0000e+00
Epoch 2/50
8/8 [==============================] - 6s 766ms/step - loss: 239.9790 - accuracy: 0.0000e+00 - val_loss: 302.4931 - val_accuracy: 0.0000e+00
Epoch 3/50
8/8 [==============================] - 6s 754ms/step - loss: 257.3013 - accuracy: 0.0000e+00 - val_loss: 280.1821 - val_accuracy: 0.0000e+00
Epoch 4/50
8/8 [==============================] - 6s 743ms/step - loss: 265.8564 - accuracy: 0.0000e+00 - val_loss: 334.2436 - val_accuracy: 0.0000e+00
Epoch 5/50
8/8 [==============================] - 6s 761ms/step - loss: 245.2486 - accuracy: 0.0000e+00 - val_loss: 190.3346 - val_accuracy: 0.0000e+00
Epoch 6/50
8/8 [==============================] - 6s 725ms/step - loss: 256.9097 - accuracy: 0.0000e+00 - val_loss: 341.5477 - val_accuracy: 0.0000e+00
Epoch 7/50
8/8 [==============================] - 6s 775ms/step - loss: 245.3100 - accur